# Pandas & Strings

This notebook explores how to work with strings in Pandas, focusing on common tasks such as how to select information from inside strings and filter rows in a dataframe based on the text in a particular column. 

The notebook assumes a baseline knowledge of Python and Pandas, as well as some familiarity with regular expressions. If you're mostly happy navigating dataframes filled with numbers, but you want to be equally comfortable working with more complex (and interesting data), then this is the notebook for you.

For no especial reason, the data used to illustrate the code in this notebook is the full text of Mary Wollstonecraft Shelley's [*Frankenstein*](https://www.gutenberg.org/ebooks/42324). 

## Imports

Pandas is required because the notebooks is about Pandas; all other libraries are simply used for initial data preparation.

In [1]:
import pandas as pd  # Data manipulation
import re  # Regular expressions
from nltk.tokenize import sent_tokenize  # Split the raw text into sentences

In [2]:
# Set the pandas display width to show twice as much text as normal

pd.set_option("display.max_colwidth", 100)

## Data sourcing

The data is stored in a text file, and so requires a small amount of processing to fit into a Pandas' object. The end result is a Pandas `Series` containing the full text, one sentence per row.

In [3]:
# Load the data from a file

with open("frankenstein.txt", "r") as file:
    raw_text = file.read()
    
# Necessary cleaning

raw_text = re.sub("\n", " ", raw_text)  # Change newlines to spaces
raw_text = re.sub("_", "", raw_text)  # Remove underscores (used for italics in the text)
raw_text = re.sub("\s+", " ", raw_text)  # Collapse multiple spaces
        
# Convert the raw_text into a series of sentences
# Split the text into sentences with sent_tokenize, pass into pd.Series

frankenstein = pd.Series(sent_tokenize(raw_text))

In [4]:
# How many sentences?

frankenstein.shape

(3328,)

In [5]:
# Inspect the series

frankenstein.head()

0                                                               FRANKENSTEIN; OR, THE MODERN PROMETHEUS.
1                                                                                              LETTER I.
2                                                                              To Mrs. Saville, England.
3                                                                      St. Petersburgh, Dec. 11th, 17--.
4    You will rejoice to hear that no disaster has accompanied the commencement of an enterprise whic...
dtype: object

## Datatypes

A Pandas `Series`, or a column in a larger dataframe (also a `Series`) which contains strings will have a type of `object`; this is not an error, but a general type that Pandas uses to store strings and various other values. Most non-numeric columns in a dataframe will appear to be `object`s.

In [6]:
# What type is the series?

frankenstein.dtype

# 'O' is for 'Object'

dtype('O')

## Converting series to strings

By default, Pandas will import strings correctly as `object` columns/series. However, this doesn't always mean that every single value in the series will really be have a `str` type; remember that `object` is the catch-all category. NaN values, for example, can appear in object columns, but will actually have a type of `float`. This can cause problems when you attempt to treat an `object` column as containing only strings.

To convert a column (or set of columns) to strings explicitly, use `.astype(str)`. This will force each value in the column(s) to be a string. Note that the data type of the column will still be `object`.

In this case, our data is already in string format, but it's a trick worth knowing.

In [7]:
# Ensuring that all values are strings (not required in this case)

frankenstein = frankenstein.astype(str)

## Accessing string methods

When working with strings in Pandas, you'll often want to apply the same built-in methods as you would with an individual string. However, you can't directly call string methods on Pandas `Series` objects, because only actual strings have those methods, and a series - even a series containing just strings - has its own unrelated methods instead. Think of a series as just a box you can put anything in - you have to deal with the box first, and can't skip straight to the contents.

Pandas does provide a way to access those methods though; you can access string methods for values in a series by using `.str.` and then the method you want to apply to the strings. Using `.str.` like this applies the given method to each string in turn, just as calling `.apply()` does for other methods.

In [8]:
frankenstein.str.lower().head()

0                                                               frankenstein; or, the modern prometheus.
1                                                                                              letter i.
2                                                                              to mrs. saville, england.
3                                                                      st. petersburgh, dec. 11th, 17--.
4    you will rejoice to hear that no disaster has accompanied the commencement of an enterprise whic...
dtype: object

The code cell above returns a copy of the `frankenstein` series, but with each value converted to lowercase. In this way, you can apply any string method to a column containing strings.

`.str.` is not limited to only the methods that you could use with a normal string in Python; there are some extra functions that you have access to using Pandas.

## Contains

With Python, the easiest way to check if a string contains a particular set of characters is with the `in` keyword.

In [9]:
"mode" in "the modern prometheus" 

True

`in` cannot be used with a series in the same way that it can be used with strings. However, Pandas provides another method - `.str.contains()` - which can be used to check if each string in a column contains another.

In [10]:
frankenstein.str.contains("Victor")

0       False
1       False
2       False
3       False
4       False
        ...  
3323    False
3324    False
3325    False
3326    False
3327    False
Length: 3328, dtype: bool

The above code returns a series of boolean values, based on whether each separate value in the series contains the substring "Victor". `.str.contains()` can also be used with regular expressions to pick out a more general pattern. The code below returns a boolean series checking if each sentence contains any number of digits.

In [11]:
frankenstein.str.contains("\d+", regex=True)

0       False
1       False
2       False
3        True
4       False
        ...  
3323    False
3324    False
3325    False
3326    False
3327    False
Length: 3328, dtype: bool

## Filtering with strings

You can use string columns to filter and subset a dataframe just as you would use any other column. Many of the `.str.` methods return a boolean series that can be used to directly filter rows using the familiar syntax.

`.str.contains()` produces such a boolean series, and so you can filter for "rows that contain a particular pattern/word". The code below returns all the lines that contain any digit, building on the previous example.

In [12]:
frankenstein[frankenstein.str.contains("\d+", regex=True)]

3                                                                         St. Petersburgh, Dec. 11th, 17--.
54                                                                             Archangel, 28th March, 17--.
108                                                                         MY DEAR SISTER, July 7th, 17--.
128                                                                                       August 5th, 17--.
130     Last Monday (July 31st), we were nearly surrounded by ice, which closed in the ship on all sides...
187                                                                                      August 13th, 17--.
223                                                                                              August 19.
224                                                                                                   17--.
646                                                       "[1] [Footnote 1: Coleridge's "Ancient Mariner."]
778                         

`.str.contains()` is not the only useful method for filtering. You can also, for example, use `.str.startswith()` and `.str.endswith()` to filter out just those rows that begin or end in a particular way.

The example below selects all the rows that end with a question mark. Note that this method **does not accept regular expressions**, and only matches exactly - a sentence ending with `?"` would not be returned by this filter.

In [13]:
frankenstein[frankenstein.str.endswith("?")]

7                                                                  Do you understand this feeling?
14                                         What may not be expected in a country of eternal light?
35                      And now, dear Margaret, do I not deserve to accomplish some great purpose?
45                       I do not intend to sail until the month of June; and when shall I return?
46                                                Ah, dear sister, how can I answer this question?
                                                   ...                                            
3288                                                               Was there no injustice in this?
3289                  Am I to be thought the only criminal, when all human kind sinned against me?
3290                 Why do you not hate Felix, who drove his friend from his door with contumely?
3291            Why do you not execrate the rustic who sought to destroy the saviour of his child?
3312    Po

## Extract

Another useful method is `.str.extract()`; rather than checking **if** a string contains a particular sequence, it extracts the text that matches a particular pattern. This is overkill if you're just checking an exact sequence: use `str.contains()` in that case. However, if you are looking to get information of a specific type from a string (telephone numbers, product codes, colours of clothing), then combining `.str.extract()` with a regular expression is the neatest way to get the information that you want.

The method works by looking for **capturing groups**; it's not just looking for a pattern, but for the bit of the pattern you told it mattered. In regular expressions, you define a capturing group with round brackets. A pattern without these round brackets will fail, because Pandas won't know which bit of the pattern you actually want to get back.

The line below shows the usage of `.str.extract()` with a capturing group. The pattern says to look for a number - of any length - and wraps that number in round brackets as a capturing group so that `.str.extract()` knows to focus on it. Any row containing a number returns that number; all others return `NaN`.

In [14]:
frankenstein.str.extract("(\d+)")

,0
0,NaN
1,NaN
2,NaN
3,11
4,NaN
...,...
3323,NaN
3324,NaN
3325,NaN
3326,NaN


By default, this method returns a dataframe; you can collapse this with the parameter `expand=False`.

You can also search for more than one capturing group at at time. The code below searches for two groups at once; note that this returns one column for each group, and that `expand=False` will no longer force the result into a series.

In [15]:
# Add .dropna() on the end just to only show interesting rows

frankenstein.str.extract("([A-Z][a-z]+) ([A-Z][a-z]+)").dropna().head(10)

,0,1
2,To,Mrs
21,North,Pacific
32,North,Sea
53,To,Mrs
94,Ancient,Mariner
107,To,Mrs
127,To,Mrs
130,Last,Monday
153,Good,God
225,Captain,Walton


The pattern used in the example above is both crude and complex. It probably requires some explanation.

Essentially, there are two parts to it, each one in its own capturing group, and separated from the other by a space. The two parts are identical.

`[A-Z][a-z]+` searches for a capital letter followed by any number of lowercase letters. Essentially, this searches for any word beginning with a capital letter.

As a whole, the pattern searches for a word beginning with a capital letter, then a space, then another word beginning with a capital letter. Only the words themselves are in capturing groups, so they are returned as the results of the `.str.extract()` method, discarding the space.

This is a relatively blunt method for searching for names (of people or places) within the text. You can see that it has mostly failed, particularly where a name started the second word of a sentence ("To Mrs. Saville"), matching the pattern correctly but not providing the desired information. However, it has picked up on some names: "Captain Walton", "North Sea".

It's not a great regex, and could do with a lot of refining, but hopefully the basic idea is clear.

## Replace

You can always use the `.replace()` method in Pandas to work with strings, but it's worth being aware of the `.str.replace()` method as well. The primary difference is that - by default - `.replace()` searches for and replaces whole values; if you want regular expressions, you need to pass in `regex=True`. `.str.replace()` accepts regex patterns by default, and gives you lots of potential arguments to fine-tune the replacements. It's a little more fiddly, but if you want to do something more complex, it's worth considering `.str.replace()`.

The code below searches the series for any sentence which mentions "the monster", "the fiend", or "the devil"; all of these are names that Frankenstein bestows on his innocent creation, and if you've read the novel, you find that as unjust as I do. To correct this historic injustice, we're going to replace all incidents of the above insults with "Adam". Although the creature is never named in the original novel, Shelley makes the suggestion in the creature's impassioned reproach to his father's rejection:


> "Remember, that I am thy creature; I ought to be thy Adam; but I am rather the fallen angel, whom thou drivest from joy for no misdeed."

In [16]:
# Filter for just lines that contain "the monster", "the devil", or "the fiend"
# using capturing groups and the pipe - | for "or"

monster_series = frankenstein[frankenstein.str.contains("the (monster|fiend|devil)")]

/usr/local/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


You may get a pink warning from the above cell, because you're using capturing groups to organise the information, but not actually requesting the group's return; this message can be safely ignored.

In [17]:
monster_series.head()

682                   I imagined that the monster seized me; I struggled furiously, and fell down in a fit.
692     The form of the monster on whom I had bestowed existence was for ever before my eyes, and I rave...
946     I thought of pursuing the devil; but it would have been in vain, for another flash discovered hi...
1134    Ever since I was condemned, my confessor has besieged me; he threatened and menaced, until I alm...
1139    She paused, weeping, and then continued--"I thought with horror, my sweet lady, that you should ...
dtype: object

In [18]:
# Replace the insults with the name he deserves

monster_series.str.replace("the (monster|fiend|devil)", "Adam").head()

682                          I imagined that Adam seized me; I struggled furiously, and fell down in a fit.
692     The form of Adam on whom I had bestowed existence was for ever before my eyes, and I raved inces...
946     I thought of pursuing Adam; but it would have been in vain, for another flash discovered him to ...
1134    Ever since I was condemned, my confessor has besieged me; he threatened and menaced, until I alm...
1139    She paused, weeping, and then continued--"I thought with horror, my sweet lady, that you should ...
dtype: object

## Conclusions

The methods discussed above should allow you to meet most common use cases for working with strings: checking if they contain information, grabbing the information they do contain, and fixing incorrect information. There are lots of subtleties and complexities that I recommend exploring if you're interested, but 99% of the time, these are the methods you need to wrangle (not analyse) text data.